In [2]:
import findspark

In [3]:
findspark.init('/home/samet/spark-3.2.1-bin-hadoop3.2')

In [4]:
from pyspark.sql import SparkSession

In [7]:
spark = SparkSession.builder.appName('titanic').getOrCreate()

In [8]:
from pyspark.ml.classification import LogisticRegression

In [9]:
data = spark.read.csv('titanic.csv', inferSchema=True, header=True)

In [13]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [14]:
for item in data.head(1)[0]:
    print(item)

1
0
3
Braund, Mr. Owen Harris
male
22.0
1
0
A/5 21171
7.25
None
S


In [19]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [20]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [16]:
my_cols = data.select(['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked'])

In [17]:
my_final_data = my_cols.na.drop()

In [21]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [22]:
gender_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndex')

gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')

In [23]:
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [24]:
assembler = VectorAssembler(inputCols=['Pclass',
 'SexVec',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'EmbarkVec'],outputCol='features')

In [25]:
from pyspark.ml.classification import LogisticRegression

In [26]:
from pyspark.ml import Pipeline

In [28]:
log_reg_titanic = LogisticRegression(featuresCol='features', labelCol='Survived')

In [29]:
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,
                           gender_encoder,embark_encoder,
                           assembler,log_reg_titanic])

In [30]:
train_data, test_data = my_final_data.randomSplit([0.7, 0.3])

In [33]:
fit_model = pipeline.fit(train_data)

In [34]:
results = fit_model.transform(test_data)

In [35]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [36]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

In [37]:
results.select('Survived', 'prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [39]:
AUC = my_eval.evaluate(results)

In [40]:
AUC

0.7533150684931508